In [30]:
#-*-coding:utf8-*-

from __future__ import division #导入精确除法
import os
import sys
import numpy as np
import pandas as pd
import operator
from scipy.sparse import coo_matrix #引入稀疏矩阵模块
from scipy.sparse.linalg import gmres#用于解Ax=b这一类的线性方程


### 通过movies.csv获取电影信息

In [31]:
def get_item_info(input_file):
    if not os.path.exists(input_file):
        return {}
    item_info={}
    linenum=0
    fp = open(input_file,encoding='UTF-8')
    for line in fp:
        if linenum == 0:
            linenum += 1
            continue
        item = line.strip().split(',')
        if len(item)<3:
            continue
        elif len(item) == 3:
            itemid,title,genre = item[0],item[1],item[2]
        elif len(item)>3:
            itemid = item[0]
            genre = item[-1]
            title = ','.join(item[1:-1])
        item_info[itemid]=[title,genre]
    fp.closed
    return item_info

### 图算法的数据格式

In [32]:
def get_graph_from_data(input_file):
    """
    Args:
        input_file:user item rating file
    Return:
        a dict:{User A:{itemb:1,itemc:1},itemb:{UserA:1}}
    """
    if not os.path.exists(input_file):
        return {}   
    graph={}
    linenum =0
    score_thr=4.0
    fp = open(input_file)
    for line in fp:
        if linenum ==0:
            linenum +=1
            continue
        item = line.strip().split(",")
        if len(item)<3:
            continue
        userid,itemid,rating =item[0],"item_"+item[1],item[2]
        if float(rating)<score_thr:
            continue
        if userid not in graph:
            graph[userid] ={}
        graph[userid][itemid]=1
        if itemid not in graph:
            graph[itemid]={}
        graph[itemid][userid] = 1
    fp.close()
    return graph

In [33]:
graph=get_graph_from_data("../data/ratings15000.csv")
# graph

### 将图转成M矩阵

In [34]:
def graph_to_m(graph):
    """
    Arges:   
        graph:二分图
    Return:
       a coo_matrix sparse mat M 稀疏矩阵
       a list ,total user item point
       a dict ,map all the point to row index
    """
    vertex = graph.keys()#所有的顶点
    address_dict = {}#记录所有顶点位置的数据结构
    total_len = len(vertex)
    for index in range(len(vertex)):
#  address_dict[vertex[index]]=index      #注意这里是python2和python3不同书写的地方
        address_dict.update({list(vertex)[index]:index})#知道了每一行对应哪个顶点
    row = []
    col = []
    data = []
    for element_i in graph:
        #顶点i到顶点j有没有连通，如果有就是顶点i出度的倒数
        weight =round(1/len(graph[element_i]),3)
        row_index =  address_dict[element_i] #i顶点对应的列索引
        for  element_j in graph[element_i]:
            col_index = address_dict[element_j]
            row.append(row_index)
            col.append(col_index)
            data.append(weight)
    row = np.array(row)     
    col = np.array(col)
    data =np.array(data)
    m = coo_matrix((data,(row,col)),shape=(total_len,total_len))
    return m,vertex,address_dict

In [35]:
def mat_all_point(m_mat,vertex,alpha):
    """
    get E - alpha*m_mat.T
    Args:
        m_mat:单位阵
        vertex:total item and user point M矩阵
        alpha:the prob for random walking随机游走的概率 
    Return:
        a sparse
    """
    total_len =len(vertex)#单位阵和M矩阵行列数相等
    row =[]
    col =[]
    data=[]
    for index in range(total_len):
        row.append(index)
        col.append(index)
        data.append(1)
    row =np.array(row)
    col =np.array(col)
    data =np.array(data)
    eye_t=  coo_matrix ((data,(row,col)),shape=(total_len,total_len))#构建单位阵结束
    #输出格式使用csv格式这样可以使运算变得快速一点，
    return eye_t.tocsr() - alpha*m_mat.tocsr().transpose()

In [36]:
graph=get_graph_from_data("../data/log.txt")
m,vertex,address_dict = graph_to_m(graph)
print(mat_all_point(m,vertex,8).todense())
print(address_dict)
print(m.todense())
print(m)

[[ 1.    -4.    -4.    -4.     0.     0.     0.     0.     0.   ]
 [-2.664  1.     0.     0.    -4.     0.     0.     0.     0.   ]
 [-2.664  0.     1.     0.     0.     0.    -4.     0.     0.   ]
 [-2.664  0.     0.     1.     0.     0.     0.     0.    -4.   ]
 [ 0.    -4.     0.     0.     1.    -4.     0.     0.     0.   ]
 [ 0.     0.     0.     0.    -4.     1.     0.     0.    -4.   ]
 [ 0.     0.    -4.     0.     0.     0.     1.    -8.     0.   ]
 [ 0.     0.     0.     0.     0.     0.    -4.     1.     0.   ]
 [ 0.     0.     0.    -4.     0.    -4.     0.     0.     1.   ]]
{'A': 0, 'item_a': 1, 'item_b': 2, 'item_d': 3, 'B': 4, 'item_c': 5, 'C': 6, 'item_e': 7, 'D': 8}
[[0.    0.333 0.333 0.333 0.    0.    0.    0.    0.   ]
 [0.5   0.    0.    0.    0.5   0.    0.    0.    0.   ]
 [0.5   0.    0.    0.    0.    0.    0.5   0.    0.   ]
 [0.5   0.    0.    0.    0.    0.    0.    0.    0.5  ]
 [0.    0.5   0.    0.    0.    0.5   0.    0.    0.   ]
 [0.    0.    0.    0.

### 传统的personalRank算法模型

In [37]:
def personal_rank(graph,root,alpha,iter_num,recom_num=10):
    """
    Args:
        graph:user item graph
        root:指定要推荐的用户
        alpha：以alpha的概率选择随机游走，以1-alpha的概率回到起点
        item_num:迭代轮次
        recom_num=10:指定迭代轮次
    Return:
        
    """
    rank = {}
    rank = {point:0 for point in graph}#将除了root顶点以外，其他所有顶点初始化为0
    rank[root] = 1#root顶点初始化成1
    recom_result={}#输出的数据结构
    for iter_index in range(iter_num):
        tmp_rank = {}
        tmp_rank = {point:0 for point in graph}#该迭代轮次下其余顶点到root顶点的pr值
        #如果该顶点不是root顶点,那么所有连接该顶点的顶点的pr值以1/N的概率贡献给这个顶点
        for out_point,out_dict in graph.items():
            for inner_point,value in graph[out_point].items():
#                 如果顶点不是root顶点（公式的上半部分）
                tmp_rank[inner_point] +=round(alpha*rank[out_point]/len(out_dict),4)
#                公式的下半部分
                if inner_point == root:
                    tmp_rank[inner_point] +=round(1-alpha,4)
#         迭代充分了提前结束迭代
        if tmp_rank ==rank:
            print("out"+str(iter_index))#查看是否提前结束迭代
            break
#         如果没有完全迭代充分，就要赋值给rank这个数据结构
        rank = tmp_rank
    
    right_num = 0#定义一个计数器
    
#     将rank这个结构根据pr值的得分进行排序，并过滤掉User顶点和root顶点已经行为过的item 
    for zuhe in sorted(rank.items(),key=operator.itemgetter(1),reverse=True):
        point,pr_score =zuhe[0],zuhe[1]
        if len(point.split('_'))<2:#如果不是item顶点就过滤掉
            continue
        if point in graph[root]:#如果被root顶点行为过，同样过滤
            continue
        recom_result[point] = pr_score #结果装载进数据集
        right_num += 1
        if right_num >recom_num:
            break#迭代轮次结束
    return recom_result

### 将personalRank矩阵化算法模型

In [38]:
def personal_rank_mat(graph,root,alpha,recom_num=10):
    """
    Args:
        graph:用户 物品 二分图
        root:固定推荐的用户
        alpha:随机游走概率
        recom_num:推荐物品数目
    Return：
        a dict,key:itemid,value:pr score
    """
#     m矩阵，m矩阵所有顶点的集合，所有顶点对应的行号
    m,vertex,address_dict =graph_to_m(graph)
    if root not in address_dict:
        return {}
    score_dict ={}
    recom_dict ={}
    mat_all = mat_all_point(m,vertex,alpha)
    index = address_dict[root]#root顶点对应的行号，index的目的是为了得到r0矩阵
    initial_list =[[0] for row in range(len(vertex))]#初始化r0矩阵
    initial_list[index] =[1]
    r_zero = np.array(initial_list)
    #解线性方程得到一个元组，第一个元素是所有顶点对该root顶点的pr值得分
    res =  gmres(mat_all,r_zero,tol=1e-8)[0]

    
    for index in range(len(res)):
        
#         point = vertex[index]#这里又是python2和python3不同书写的地方
        point =list(vertex)[index]
        
        address_dict.update({list(vertex)[index]:index})#知道了每一行对应哪个顶点

        if len(point.strip().split("_"))<2:#如果不是item顶点就过滤掉
            continue
        if point in graph[root]:#如果被root顶点行为过，同样过滤
            continue
        score_dict[point] = round(res[index],3)
    for zuhe in sorted(score_dict.items(),key = operator.itemgetter(1),reverse=True):
        point,score = zuhe[0],zuhe[1]
        recom_dict[point]=score
    return recom_dict

In [39]:
def get_one_user_recom():
    """
    give one fix user recom result
    """
    user ="1"# A
    alpha = 0.8
#     graph = get_graph_from_data("../data/log.txt")
    graph =get_graph_from_data("../data/ratings15000.csv")
    iter_num = 100  
    recom_result=personal_rank(graph,user,alpha,100)
    return recom_result
#     item_info = get_item_info("../data/movies.csv")
#     将用户感兴趣的物品打印出来分析结果
#     for itemid in graph[user]:
#         pure_itemid = itemid.split("_")[1]
#         print(item_info[pure_itemid])
#     print("result------------")    
#     for itemid in recom_result:
#         pure_itemid = itemid.split("_")[1]
#         print(item_info[pure_itemid])
#         print(recom_result[itemid])    

In [40]:
def get_one_user_by_mat():
    """
    give one fix user by mat
    """
    user ="112"# A
    alpha = 0.8
#     graph = get_graph_from_data("../data/log.txt")
    graph =get_graph_from_data("../data/ratings15000.csv")
    recom_result=personal_rank_mat(graph,user,alpha,100)

    item_info = get_item_info("../data/movies.csv")
    # 将用户感兴趣的物品打印出来分析结果
    for itemid in graph[user]:
        pure_itemid = itemid.split("_")[1]
        print(item_info[pure_itemid])
    print("result------------")
    for itemid in recom_result:
        pure_itemid = itemid.split("_")[1]
        print(item_info[pure_itemid])
        print(recom_result[itemid])
    return recom_result

In [41]:
get_one_user_by_mat()

['Dead Man Walking (1995)', 'Crime|Drama']
['Ed Wood (1994)', 'Comedy|Drama']
['"Shawshank Redemption, The (1994)"', 'Crime|Drama']
['Strawberry and Chocolate (Fresa y chocolate) (1993)', 'Drama']
['"Fugitive, The (1993)"', 'Thriller']
['In the Name of the Father (1993)', 'Drama']
['"Piano, The (1993)"', 'Drama|Romance']
["Schindler's List (1993)", 'Drama|War']
['Fargo (1996)', 'Comedy|Crime|Drama|Thriller']
['Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)', 'Comedy|War']
['Rear Window (1954)', 'Mystery|Thriller']
['Monty Python and the Holy Grail (1975)', 'Adventure|Comedy|Fantasy']
['"Princess Bride, The (1987)"', 'Action|Adventure|Comedy|Fantasy|Romance']
['12 Angry Men (1957)', 'Drama']
['Psycho (1960)', 'Crime|Horror']
['Amadeus (1984)', 'Drama']
['Cool Hand Luke (1967)', 'Drama']
['Sling Blade (1996)', 'Drama']
['Waiting for Guffman (1996)', 'Comedy']
['"Truman Show, The (1998)"', 'Comedy|Drama|Sci-Fi']
['Good Will Hunting (1997)', 'Drama|Romance']
['

{'item_296': 0.009,
 'item_356': 0.008,
 'item_593': 0.007,
 'item_260': 0.006,
 'item_50': 0.006,
 'item_2858': 0.006,
 'item_1196': 0.005,
 'item_1198': 0.005,
 'item_2762': 0.005,
 'item_2959': 0.005,
 'item_110': 0.005,
 'item_589': 0.005,
 'item_858': 0.005,
 'item_2571': 0.005,
 'item_47': 0.005,
 'item_541': 0.004,
 'item_4993': 0.004,
 'item_480': 0.004,
 'item_1210': 0.004,
 'item_1': 0.004,
 'item_32': 0.004,
 'item_2028': 0.004,
 'item_2329': 0.004,
 'item_590': 0.004,
 'item_1265': 0.004,
 'item_3578': 0.004,
 'item_778': 0.004,
 'item_293': 0.003,
 'item_1036': 0.003,
 'item_1079': 0.003,
 'item_1097': 0.003,
 'item_1240': 0.003,
 'item_1258': 0.003,
 'item_4306': 0.003,
 'item_5952': 0.003,
 'item_924': 0.003,
 'item_1270': 0.003,
 'item_1580': 0.003,
 'item_1089': 0.003,
 'item_1193': 0.003,
 'item_1206': 0.003,
 'item_1208': 0.003,
 'item_1213': 0.003,
 'item_1221': 0.003,
 'item_1721': 0.003,
 'item_150': 0.003,
 'item_364': 0.003,
 'item_380': 0.003,
 'item_500': 0.00

### 对比两种算法

In [42]:
# recom_result__base = get_one_user_recom()
# recom_result__mat = get_one_user_by_mat()
# num =0
# for ele in recom_result__base:
#     if ele in recom_result__mat:
#         num +=1
# print(num) #重合度大概四分之三左右


In [45]:
lll=[
    ['"Piano, The (1993)"', 'Drama|Romance'],
['Dead Man Walking (1995)', 'Crime|Drama'],
['Ed Wood (1994)', 'Comedy|Drama'],
    ['12 Angry Men (1957)', 'Drama'],

['Strawberry and Chocolate (Fresa y chocolate) (1993)', 'Drama'],
['In the Name of the Father (1993)', 'Drama'],
["Schindler's List (1993)", 'Drama|War'],
    ['"Shawshank Redemption, The (1994)"', 'Crime|Drama'],
['Fargo (1996)', 'Comedy|Crime|Drama|Thriller'],
['Rear Window (1954)', 'Mystery|Thriller'],
['Monty Python and the Holy Grail (1975)', 'Adventure|Comedy|Fantasy'],
['"Princess Bride, The (1987)"', 'Action|Adventure|Comedy|Fantasy|Romance'],

['Psycho (1960)', 'Crime|Horror'],
['Amadeus (1984)', 'Drama'],
['Cool Hand Luke (1967)', 'Drama'],
['Sling Blade (1996)', 'Drama'],
['Waiting for Guffman (1996)', 'Comedy'],
['"Truman Show, The (1998)"', 'Comedy|Drama|Sci-Fi'],
['Good Will Hunting (1997)', 'Drama|Romance']]
for ea in lll:
    print(ea)

get_one_user_by_mat()

['"Piano, The (1993)"', 'Drama|Romance']
['Dead Man Walking (1995)', 'Crime|Drama']
['Ed Wood (1994)', 'Comedy|Drama']
['12 Angry Men (1957)', 'Drama']
['Strawberry and Chocolate (Fresa y chocolate) (1993)', 'Drama']
['In the Name of the Father (1993)', 'Drama']
["Schindler's List (1993)", 'Drama|War']
['"Shawshank Redemption, The (1994)"', 'Crime|Drama']
['Fargo (1996)', 'Comedy|Crime|Drama|Thriller']
['Rear Window (1954)', 'Mystery|Thriller']
['Monty Python and the Holy Grail (1975)', 'Adventure|Comedy|Fantasy']
['"Princess Bride, The (1987)"', 'Action|Adventure|Comedy|Fantasy|Romance']
['Psycho (1960)', 'Crime|Horror']
['Amadeus (1984)', 'Drama']
['Cool Hand Luke (1967)', 'Drama']
['Sling Blade (1996)', 'Drama']
['Waiting for Guffman (1996)', 'Comedy']
['"Truman Show, The (1998)"', 'Comedy|Drama|Sci-Fi']
['Good Will Hunting (1997)', 'Drama|Romance']
